In [ ]:
from collections import OrderedDict
from glob import glob
import numpy as np
import pandas as pd
from os.path import join
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import xarray as xr

import torch
import torch.nn as nn
from torch import Tensor
from torchmetrics.utilities.checks import _check_same_shape
from torchmetrics import Metric
import pytorch_lightning as pl
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
import pylab as plt
import matplotlib

In [ ]:
import sys
sys.path.insert(0, "../pismemulator/")

from nnemulator import PISMDataset

In [ ]:
dataset = PISMDataset(data_dir="../data/speeds_v2/",
                      samples_file="../data/samples/velocity_calibration_samples_100.csv",
                      target_file="../data/observed_speeds/greenland_obs_g1800m.nc",
                     target_corr_threshold=25)
            

In [ ]:
cd ../speedemulator/

In [ ]:
run ../speedemulator/sample_posterior.py --emulator_dir ../speedemulator/2021_11_gris_2km_cpu --data_dir ../data/speeds_v2/ --target_file ../data/observed_speeds/greenland_obs_g1800m.nc

In [ ]:
data_dir="../data/speeds_v2/"
samples_file="../data/samples/velocity_calibration_samples_100.csv"
target_file="../data/observed_speeds/greenland_obs_g1800m.nc"
target_var="velsurf_mag"
target_corr_threshold=25.0
target_corr_var="thickness"
target_error_var="velsurf_mag_error"
training_var="velsurf_mag"
thinning_factor=1
normalize_x=True
log_y=True
threshold=100e3
epsilon=0
return_numpy=False


In [ ]:
A = torch.randn(2, 2, dtype=torch.complex128)

In [ ]:
A

In [ ]:
B = torch.randn(2, 2, dtype=torch.float)

In [ ]:
B

In [ ]:
U, V = torch.linalg.eig(B)

In [ ]:
import torch
import torch.nn as nn
from torch import Tensor
from torchmetrics.utilities.checks import _check_same_shape
from torchmetrics import Metric
import pytorch_lightning as pl
from collections import OrderedDict


In [ ]:
n_hidden = [128, 256, 256, 128]
n_layers = 5
n = 0

models = []
for  n in range(n_layers - 2):
    models.append(nn.Sequential(
        OrderedDict(
            [
                ("Linear", nn.Linear(n_hidden[n], n_hidden[n+1])),
                ("LayerNorm", nn.LayerNorm(n_hidden)),
                ("Dropout", nn.Dropout(p=0.1)),
            ]
        ))
                 )

In [ ]:
nn.ModuleList(models)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
import pymc3 as pm

In [ ]:
rng = np.random.RandomState(1)
x = 10 * rng.rand(50)
y = 2 * x - 5 + 2*rng.randn(50)
x = x.reshape(-1, 1)
y = y.reshape(-1, 1)

plt.scatter(x, y)


In [ ]:
basic_model =  pm.Model()

In [ ]:
with basic_model as bm:

    #Priors
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10)
    sigma = pm.HalfNormal('sigma', sd=1)

    # Deterministics
    mu = alpha + beta*x
    
    # Likelihood 
    Ylikelihood = pm.Normal('Ylikelihood', mu=mu, sd=sigma, observed=y)

In [ ]:
trace = pm.sample(draws=2000,model=bm)

In [ ]:
pm.traceplot(trace)

In [ ]:
print(pm.summary(trace).round(2))

In [ ]:
model =  pm.Model()
with model as m:

    #Priors
    alpha = pm.Normal('alpha', mu=0, sd=10)
    beta = pm.Normal('beta', mu=0, sd=10)
    xn = pm.Normal('x', mu=0, sd=10)
    sigma = pm.HalfNormal('sigma', sd=1)

    # Deterministics
    mu = alpha + beta*xn
    
    # Likelihood 
    Ylikelihood = pm.Normal('Ylikelihood', mu=mu, sd=sigma, observed=y)

In [ ]:
intercept = 0 
gradient = 2.5
size = 10000

x_true = np.array(np.linspace(0,1,size))
y_true = intercept + gradient * x_true

# adding noise to data
x = x_true + np.random.normal(loc=0, scale=0.5, size=size)
y = y_true + np.random.normal(loc=0, scale=0.5, size=size)

In [ ]:
with pm.Model() as model:
    err_odr = pm.HalfNormal('err_odr', 5.)
    err_param = pm.HalfNormal('err_param', 5.)
    x_lat = pm.Normal('x_mu', 0, 5., shape=x.shape[0])
    x_obs = pm.Normal('x_obs', mu=x_lat, sd=err_odr, observed=x, shape=x.shape[0])
    offset = pm.Normal('offset', 0, err_param)
    slope = pm.Normal('slope', 0, err_param)
    y_pred = pm.Deterministic('y_pred', offset + slope * x_lat)
    y_obs = pm.Normal('y', mu=y_pred, sd=err_odr, observed=y)
    
    trace = pm.sample(800, tune=1500, chains=4, cores=8, init='jitter+adapt_diag')
    pm.traceplot(trace)

In [ ]:
85*1.1 * 1.1

In [ ]:
clear